## Utilizing ChatGPT to Develop High-risk Power System Scenarios in pandapower

### Objective: prompt ChatGPT to create risk scenarios for pandapower and evaluate the given code.


#### ChatGPT prompt: 
could you create five scenarios in pandapower for high risk in a power system?

#### ChatGPT responded: 
with five outlines for various problems that could occur in a power system (line overloading, generator tripping, transformer overload/failure, islanding/ loss of connectivity, short circuit fault) with short, one-sentence details and risk definition. 
It also gave a code model that could be used to initialize the risk scenarios. 

#### ChatGPT prompt: 
copied and pasted __[create github-note-book-pandapower task](https://github.com/ckwon06/C.Lee--2025-Summer-Internship-Materials/blob/master/github-notebook-pandapower.md)__ 

"here is a task I was given previously with pushing an example power system. Give me the same steps with full code except for the five risk scenario models"

#### ChatGPT Responded

In [2]:
import pandapower as pp
import pandapower.networks as pn
import pandapower.shortcircuit as sc
import matplotlib.pyplot as plt

# Scenario 1: Line Overloading Due to High Load
net1 = pn.case9()
pp.create_load(net1, bus=5, p_mw=100, q_mvar=30)  # Artificial overload
pp.runpp(net1)
print("Scenario 1: Line Loading Due to High Load")
print(net1.res_line[["loading_percent"]])
print()

# Scenario 2: Generator Tripping (Loss of Generation)
net2 = pn.case9()
net2.gen.loc[0, 'p_mw'] = 0  # Trip the main generator
pp.runpp(net2)
print("Scenario 2: Generator Tripping")
print(net2.res_bus[["vm_pu", "va_degree"]])
print()

# Scenario 3: Transformer Overload
net3 = pp.create_empty_network()
b1 = pp.create_bus(net3, vn_kv=110)
b2 = pp.create_bus(net3, vn_kv=20)
b3 = pp.create_bus(net3, vn_kv=20)
pp.create_ext_grid(net3, b1)
pp.create_transformer(net3, b1, b2, std_type="25 MVA 110/20 kV")
pp.create_line(net3, b2, b3, length_km=1, std_type="NAYY 4x150 SE")
pp.create_load(net3, b3, p_mw=22, q_mvar=5)
pp.runpp(net3)
print("Scenario 3: Transformer Overload")
print(net3.res_trafo[["loading_percent"]])
print()

# Scenario 4: Islanding (Loss of Connectivity)
net4 = pn.case9()
# Simulate islanding by opening a key switch (disconnect line 4)
pp.create_switch(net4, bus=net4.line.from_bus.at[4], element=4, et="l", closed=True)
net4.switch.loc[0, 'closed'] = False
pp.runpp(net4, init="auto")
print("Scenario 4: Islanding (Switch Open)")
print(net4.res_bus[["vm_pu", "va_degree"]])
print()

# Scenario 5: Three-Phase Fault
net5 = pn.case9()
sc.calc_sc(net5, fault="3ph", case="max")
print("Scenario 5: Short-Circuit Analysis")
print(net5.res_bus_sc[["ikss_ka", "ip_ka"]])


Scenario 1: Line Loading Due to High Load
   loading_percent
0        69.677335
1        37.521097
2        17.542376
3        29.903906
4        18.044323
5        47.242049
6        65.819927
7        21.637389
8        34.606252

Scenario 2: Generator Tripping
      vm_pu  va_degree
0  1.000000   0.000000
1  1.000000 -17.296960
2  1.000000 -10.227422
3  0.986285  -7.884573
4  0.974990 -12.858369
5  0.999605 -13.083636
6  0.980399 -17.951037
7  0.991058 -17.296960
8  0.958564 -15.195735

Scenario 3: Transformer Overload
   loading_percent
0        94.852398

Scenario 4: Islanding (Switch Open)
      vm_pu  va_degree
0  1.000000   0.000000
1  1.000000   6.106227
2  1.000000   8.451983
3  0.985773  -2.429747
4  0.973211  -2.738104
5  1.005869   5.613572
6  0.974223  -4.025492
7  0.991559   0.209125
8  0.957390  -5.607134



ValueError: short circuit apparent power s_sc_max_mva needs to be specified for external grid 
 Try: net.ext_grid['s_sc_max_mva'] = 1000

### Code Breakdown

#### Scenario 1: Line Overloading Due to High Load
**net1 = pn.case9()**

> net5 is assigned to a new IEEE 9-BUS system with pre-loaded standard buses, lines, loads, generators, etc. 

**pp.create_load(net1, bus=5, p_mw=100, q_mvar=30)  # Artificial overload**

> creates a load at bus 5 with 100 MW active power and 30 MVAr reactive power 

> Artificial overload message is due to the fact that the active and reactive power is unreasonably large. Something that could not be achieved in standard, done synthetically to intentionally overload the system.

**pp.runpp(net1)**

> runs the power flow analysis on the given 9-BUS scenario to solve for line flow, bus voltages, power injections, etc. 

**print("Scenario 1: Line Loading Due to High Load")**


**print(net1.res_line[["loading_percent"]])**


**print()**

> prints the heavy load of each line in percentage of its load capacity


#### Scenario 2: Generator Tripping (Loss of Generation)

**net2 = pn.case9()**

> loads a standard iEEE 9-BUS with pre-loaded standard buses, lines, loads, generators, etc and assigns to net2

**net2.gen.loc[0, 'p_mw'] = 0  # Trip the main generator**

> Disables first generator (sets active power to 0 MW), essentially a generator trip (shutdown)

**pp.runpp(net2)**

> runs power flow analysis on the given 9-BUS scenario

**print("Scenario 2: Generator Tripping")**



**print(net2.res_bus[["vm_pu", "va_degree"]])**



**print()**

> prints the power flow results,specifically the voltage magnitude and angle for each bus. 

#### Scenario 3: Transformer Overload
**net3 = pp.create_empty_network()**

> assigns net3 to an empty, custom network. Unlike the previous two examples, there are no pre-existing lines, buses, generators, etc. 

**b1 = pp.create_bus(net3, vn_kv=110)**

> assign b1 to a new bus with a nominal voltage of 110 kV (high voltage)

**b2 = pp.create_bus(net3, vn_kv=20)**

> assign b2 to a new bus with a nominal voltage of 20 kV (medium voltage)

**b3 = pp.create_bus(net3, vn_kv=20)**

> assign b3 to a new bus with a nominal voltage of 20 kV (medium voltage)

**pp.create_ext_grid(net3, b1)**

> creates an external connection to an outside grid at the high voltage bus (b1)

**pp.create_transformer(net3, b1, b2, std_type="25 MVA 110/20 kV")**

> creates a step-down transformer that goes from b1 to b2 with a 25 MVA capacity from 110 to 20 kV

**pp.create_line(net3, b2, b3, length_km=1, std_type="NAYY 4x150 SE")**

> creates a 1 km MV distrubution line between b2 and b3 in standard type (standard copper core, aluminum conductor, PVC insulation and sheath; 4 conductors that each have a 150 mm^2 cross-sectional area, in standard earth grounding)

**pp.create_load(net3, b3, p_mw=22, q_mvar=5)**

> adds a load at b3 that has active power of 22 MW and reactive power of 5 MVAr. (a relatively heavy load on the transformer)

**pp.runpp(net3)**

> run the power flow analysis with the new network. 

**print("Scenario 3: Transformer Overload")**
**print(net3.res_trafo[["loading_percent"]])**
**print()**

> returns power flow analysis results, specifically the percent used of the transformer's capacity.

#### Scenario 4: Islanding (Loss of Connectivity)
**net4 = pn.case9()**

> creates standard IEEE 9-BUS system with pre-existing units. 

**pp.create_switch(net4, bus=net4.line.from_bus.at[4], element=4, et="l", closed=True)**

> creates a switch at the 'from' bus of line 4, starts off closed and thus conducting, contorlling line 4, and only the line. 

**net4.switch.loc[0, 'closed'] = False**

> opens the switch and thus ceases to conduct flow, disconnecting line 4. Consequently, parts of the grid may be isolated from power (islanding). 

**pp.runpp(net4, init="auto")**

> runs the power flow, leting pandapower choose the best method to initialize

**print("Scenario 4: Islanding (Switch Open)")**

**print(net4.res_bus[["vm_pu", "va_degree"]])**

**print()**

> returns power flow results. shows bus voltage magnitudes, angles, etc. 

#### Scenario 5: Three-Phase Fault

**net5 = pn.case9()**

> assigns net5 to a standard IEEE 9-BUS power system

**sc.calc_sc(net5, fault="3ph", case="max")**

> attempts to run a 3-phase short circuit calculation. using the strongest condiiton

**print("Scenario 5: Short-Circuit Analysis")**


**print(net5.res_bus_sc[["ikss_ka", "ip_ka"]])**

> attempts to return symmetrical short-circuit current at each bus and the peak short-circuit current. 